In [10]:
%load_ext autoreload
%autoreload 2

import polars as pl
from data_api import get_raw, get_raw_lazy,parse_datetime
import pyarrow.dataset as ds

gcs_nline = "gs://nline-public-data"



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
df = get_raw("ghana/gridwatch_data/full_period/", "2019-10-01 00:00", "2019-10-01 01:00")

KeyboardInterrupt: 

: 

In [ ]:
df.collect()

In [ ]:
from pyarrow.dataset import dataset
import gcsfs
import polars as pl
import pandas as pd
import polars as pl
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pyarrow.fs import GcsFileSystem

cloudfs = gcsfs.GCSFileSystem()
source = "gs://nline-public-data/ghana/gridwatch_data/full_period/*.parquet"
# reference multiple parquet files

start, end ="2019-10-01 00:00", "2019-10-01 01:00"
start = parse_datetime(start)
end = parse_datetime(end)

# load efficiently into polars
ldf = pl.scan_parquet(source)
ldf = ldf.filter((pl.col("time") >= start) & (pl.col("time") < end))
ldf.collect()

In [ ]:

def get_raw(bucket_name, start_time, end_time):
    source = f"{gcs_nline}/{bucket_name}"
    
    # Convert start_time and end_time to datetime objects
    start = parse_datetime(start_time)
    end = parse_datetime(end_time)

    dataset = ds.dataset(source, partitioning="hive")
    
    df = pl.scan_pyarrow_dataset(
        dataset
    )

    df_filtered = df.filter((pl.col("time") >= start) & (pl.col("time") < end))


    return df

# Function to write filtered data to disk
def write_filtered_data(df, output_path):
    df.sink_parquet(output_path)

# Usage example
bucket_name = "ghana/gridwatch_data/full_period/"
start_time = "2019-10-01 00:00"
end_time = "2019-10-01 01:00"
output_path = "filtered_data.parquet"

df = get_raw(bucket_name, start_time, end_time)
# write_filtered_data(df, output_path)


In [ ]:
df.head(10)

In [ ]:
df.collect()

: 

In [ ]:
import dask.dataframe as dd
from datetime import datetime

def get_raw_dask(bucket_name, start_time, end_time):
    source = "gs://nline-public-data/ghana/gridwatch_data/full_period/*.parquet"

    # source = f"{gcs_nline}/{bucket_name}/*.parquet"
    
    # Convert start_time and end_time to datetime objects
    start = parse_datetime(start_time)
    end = parse_datetime(end_time)

    # Read the Parquet dataset
    df = dd.read_parquet(source, engine='pyarrow')

    # Apply time filter
    # df_filtered = df[(df.time >= start) & (df.time < end)]
    print(df.columns)
    df_filtered = df[(df['time'] >= start) & (df['time'] < end)]

    return df_filtered

def write_filtered_data_dask(df, output_path):
    df.to_parquet(output_path)

# Usage example
bucket_name = "ghana/gridwatch_data/full_period/"
start_time = "2019-10-01 00:00"
end_time = "2019-10-01 01:00"
output_path = "filtered_data_dask.parquet"

df = get_raw_dask(bucket_name, start_time, end_time)

Index(['time', 'location_longitude', 'region', 'shield_id', 'active',
       'site_geom', 'location_comment', 'location_latitude', 'district',
       'alternate_site_id', 'site_longitude', 'core_id', 'experimental_group',
       'site_id', 'respondent_id', 'site_latitude', 'was_moved',
       'receive_time', 'is_powered', 'state_of_charge', 'cell_voltage',
       'millis', 'last_unplug_millis', 'last_plug_millis', 'cellular_rssi',
       'cellular_quality', 'sd_present', 'firmware_version',
       'particle_firmware_revision', 'frequency', 'voltage', 'roll', 'pitch'],
      dtype='object')


In [ ]:
pandas_df = df.compute()

KeyboardInterrupt: 